# ネット広告のクリック予測

以下のサイトを参照\
https://signate.jp/competitions/132#abstract

## 必要なライブラリをインポート

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
# ジュピターノートブック上でグラフを表示させるための処理
%matplotlib inline

# 他に必要なライブラリは随時ここに追加する。

## データセットの取得

In [6]:
train = pd.read_table('train.tsv')
test = pd.read_table('test.tsv')
sample = pd.read_csv('sample_submit.csv', header=None)

## 特徴量を選択する。

In [29]:
# トレーニングデータ、テストデータは共に1週間分のログデータ
## データは「1行につき1回の広告表示」を意味する。
### C1~C6はカテゴリー値を数値化したもの
train.head()

,id,click,I1,I2,C1,C2,C3,C4,C5,C6,...,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14
0,0,0,5,20,421256035,2377199148,2696211399,2298910535,3133252034,1509930964,...,0,0,0,0,0,0,NaN,NaN,NaN,780.0
1,1,0,5,20,2581703001,2589684548,1998340283,2411205997,3202148071,1238839469,...,1,1,0,0,0,0,NaN,NaN,NaN,0.0
2,2,0,5,20,2581703001,1862037199,2448089184,2411205997,1107442947,3668889797,...,0,0,0,0,1,0,NaN,NaN,NaN,117.0
3,3,0,5,20,421256035,3874378935,1658216424,2298910535,2321672976,1509930964,...,0,0,0,0,0,0,6.0,6.0,34501.0,390.0
4,4,1,5,20,421256035,3874378935,1892769125,2411205997,2321672976,1509930964,...,0,0,0,0,0,0,1.0,31.0,12557.0,390.0


In [23]:
train.iloc[:, 2:18]

,I1,I2,C1,C2,C3,C4,C5,C6,I3,I4,I5,I6,I7,I8,I9,I10
0,5,20,421256035,2377199148,2696211399,2298910535,3133252034,1509930964,1,0,0,0,0,0,0,0
1,5,20,2581703001,2589684548,1998340283,2411205997,3202148071,1238839469,0,2,1,1,0,0,0,0
2,5,20,2581703001,1862037199,2448089184,2411205997,1107442947,3668889797,2,1,0,0,0,0,1,0
3,5,20,421256035,3874378935,1658216424,2298910535,2321672976,1509930964,1,0,0,0,0,0,0,0
4,5,20,421256035,3874378935,1892769125,2411205997,2321672976,1509930964,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169037,4,19,421256035,3874378935,1892769125,2411205997,2321672976,1509930964,1,0,0,0,0,0,0,0
1169038,4,19,421256035,3874378935,98956388,1332932160,2321672976,1509930964,1,0,0,0,0,0,0,1
1169039,4,19,421256035,3874378935,98956388,1332932160,2321672976,1509930964,1,0,0,0,0,0,0,0
1169040,4,19,3651044092,1862037199,2448089184,2411205997,4028731966,2068230268,1,1,0,0,0,0,0,0


In [27]:
test.iloc[:, 1:17]

,I1,I2,C1,C2,C3,C4,C5,C6,I3,I4,I5,I6,I7,I8,I9,I10
0,5,20,421256035,3874378935,1892769125,2411205997,2321672976,1509930964,1,0,0,0,0,0,0,0
1,5,20,421256035,1537671376,575044823,2298910535,1301721902,3111151614,2,0,0,1,0,0,0,0
2,5,20,421256035,1392597759,2522517098,2298910535,2321672976,1509930964,2,0,0,0,0,0,0,0
3,5,20,2581703001,1862037199,2448089184,2411205997,476085660,2068230268,1,1,0,0,0,0,0,1
4,5,20,421256035,3874378935,1892769125,2411205997,2321672976,1509930964,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295081,4,19,421256035,3874378935,1892769125,2411205997,476085660,2068230268,1,0,0,0,0,1,0,1
1295082,4,19,2686811630,2589684548,1998340283,2411205997,1238839469,2068230268,0,2,0,0,0,0,1,0
1295083,4,19,421256035,3874378935,98956388,1332932160,2321672976,1509930964,1,0,0,0,0,0,1,0
1295084,4,19,421256035,1088910726,3862505502,2298910535,2321672976,1509930964,1,0,0,0,0,0,1,0


### trainの行数と列数を確認する。

In [8]:
train.shape

(1169042, 22)

### trainの基礎統計量を見る。

In [9]:
# describe関数は数値のデータのみ、様々な計算結果を表示する。
## stdは標準偏差、50%は中央値を表している
train.describe()

,id,click,I1,I2,C1,C2,C3,C4,C5,C6,...,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14
count,1.169042e+06,1.169042e+06,1.169042e+06,1.169042e+06,1.169042e+06,1.169042e+06,1.169042e+06,1.169042e+06,1.169042e+06,1.169042e+06,...,1.169042e+06,1.169042e+06,1.169042e+06,1.169042e+06,1.169042e+06,1.169042e+06,981754.000000,981754.000000,9.817540e+05,1.129422e+06
mean,5.845205e+05,7.505889e-02,3.031627e+00,1.220570e+01,1.169567e+09,2.381345e+09,2.012817e+09,2.177769e+09,1.888389e+09,2.186933e+09,...,3.292790e-01,1.047524e-01,5.746586e-02,8.747077e-02,3.626439e-01,3.801831e-01,365.767723,14.166252,8.158472e+05,3.624405e+02
std,3.374735e+05,2.634865e-01,1.992983e+00,6.139743e+00,1.087105e+09,1.099911e+09,1.195821e+09,4.357866e+08,9.501573e+08,9.617819e+08,...,5.794303e+00,3.062343e-01,2.327307e-01,2.825238e-01,4.807635e-01,4.854319e-01,2472.558968,86.252650,1.385875e+06,6.203855e+02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.212560e+08,5.393295e+07,2.200363e+07,4.001228e+08,2.446028e+07,1.477335e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,-1.000000e+00,0.000000e+00
25%,2.922602e+05,0.000000e+00,1.000000e+00,7.000000e+00,4.212560e+08,1.537671e+09,1.025908e+09,2.298911e+09,1.410458e+09,1.509931e+09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000,1.000000,2.126325e+04,3.900000e+01
50%,5.845205e+05,0.000000e+00,3.000000e+00,1.300000e+01,4.212560e+08,1.862037e+09,1.998340e+09,2.411206e+09,2.321673e+09,1.769723e+09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,19.000000,4.000000,1.416105e+05,2.600000e+02
75%,8.767808e+05,0.000000e+00,5.000000e+00,1.700000e+01,2.581703e+09,3.874379e+09,2.903162e+09,2.411206e+09,2.321673e+09,3.111152e+09,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,84.000000,11.000000,9.316495e+05,3.900000e+02
max,1.169041e+06,1.000000e+00,6.000000e+00,2.300000e+01,3.651044e+09,4.218832e+09,4.231334e+09,3.954818e+09,4.272797e+09,4.161523e+09,...,1.965000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,58684.000000,32664.000000,1.809308e+07,1.300000e+04


### testの基礎統計量を見る。

In [10]:
test.describe()

,id,I1,I2,C1,C2,C3,C4,C5,C6,I3,...,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14
count,1.295086e+06,1.295086e+06,1.295086e+06,1.295086e+06,1.295086e+06,1.295086e+06,1.295086e+06,1.295086e+06,1.295086e+06,1.295086e+06,...,1.295086e+06,1.295086e+06,1.295086e+06,1.295086e+06,1.295086e+06,1.295086e+06,1.065379e+06,1.065379e+06,1.065379e+06,1.253651e+06
mean,1.816584e+06,3.013113e+00,1.229005e+01,1.197618e+09,2.392273e+09,2.021446e+09,2.183931e+09,1.845995e+09,2.215456e+09,1.276350e+00,...,3.526669e-01,8.314274e-02,4.858442e-02,7.209405e-02,3.473862e-01,3.712232e-01,3.901976e+02,1.320977e+01,8.053019e+05,3.704173e+02
std,3.738593e+05,1.992191e+00,6.023858e+00,1.097777e+09,1.109099e+09,1.180924e+09,4.341494e+08,9.639804e+08,9.773575e+08,6.580851e-01,...,7.664042e+00,2.760980e-01,2.149977e-01,2.586437e-01,4.761399e-01,4.831322e-01,2.701358e+03,8.150263e+01,1.390526e+06,6.253552e+02
min,1.169042e+06,0.000000e+00,0.000000e+00,4.212560e+08,4.970256e+07,2.200363e+07,4.001228e+08,2.446028e+07,1.477335e+08,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00
25%,1.492813e+06,1.000000e+00,7.000000e+00,4.212560e+08,1.537671e+09,1.276636e+09,2.298911e+09,1.238839e+09,1.509931e+09,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,1.000000e+00,1.964850e+04,3.900000e+01
50%,1.816584e+06,3.000000e+00,1.300000e+01,4.212560e+08,1.862037e+09,1.998340e+09,2.411206e+09,2.321673e+09,1.769723e+09,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+01,3.000000e+00,1.226190e+05,2.600000e+02
75%,2.140356e+06,5.000000e+00,1.700000e+01,2.581703e+09,3.874379e+09,2.780866e+09,2.411206e+09,2.321673e+09,3.111152e+09,2.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,8.700000e+01,1.000000e+01,1.009804e+06,3.900000e+02
max,2.464127e+06,6.000000e+00,2.300000e+01,3.651044e+09,4.218832e+09,4.293804e+09,3.954818e+09,4.272797e+09,4.161523e+09,2.000000e+00,...,5.720000e+02,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,7.465800e+04,1.449100e+04,1.603475e+07,1.300000e+04


⬆︎トレーニングデータセットとテストデータセットの値にそこまで大きな差は無さそう。

### データの型を確認する。

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169042 entries, 0 to 1169041
Data columns (total 22 columns):
id       1169042 non-null int64
click    1169042 non-null int64
I1       1169042 non-null int64
I2       1169042 non-null int64
C1       1169042 non-null int64
C2       1169042 non-null int64
C3       1169042 non-null int64
C4       1169042 non-null int64
C5       1169042 non-null int64
C6       1169042 non-null int64
I3       1169042 non-null int64
I4       1169042 non-null int64
I5       1169042 non-null int64
I6       1169042 non-null int64
I7       1169042 non-null int64
I8       1169042 non-null int64
I9       1169042 non-null int64
I10      1169042 non-null int64
I11      981754 non-null float64
I12      981754 non-null float64
I13      981754 non-null float64
I14      1129422 non-null float64
dtypes: float64(4), int64(18)
memory usage: 196.2 MB


### trainの各値が欠損値か否かを出力する。
- 欠損とは何らかの理由で、データの値が入っていない状態のことをいいます
- データ分析において欠損がある場合の対応はとても重要となります
- 欠損の有無を確認する為にはisnull関数を使います
- 欠損値の時、Trueと表示されます

In [13]:
# 欠損地の時にTrueになる。
train.isnull()

,id,click,I1,I2,C1,C2,C3,C4,C5,C6,...,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169037,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1169038,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1169039,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1169040,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,True,False


In [14]:
# 1つ以上欠損値を含んでいる場合、Trueになる。
train.isnull().any()

id       False
click    False
I1       False
I2       False
C1       False
C2       False
C3       False
C4       False
C5       False
C6       False
I3       False
I4       False
I5       False
I6       False
I7       False
I8       False
I9       False
I10      False
I11       True
I12       True
I13       True
I14       True
dtype: bool

In [15]:
# 各列（カラム）に欠損値が幾つあるかを数える。
train.isnull().sum()

id            0
click         0
I1            0
I2            0
C1            0
C2            0
C3            0
C4            0
C5            0
C6            0
I3            0
I4            0
I5            0
I6            0
I7            0
I8            0
I9            0
I10           0
I11      187288
I12      187288
I13      187288
I14       39620
dtype: int64

### 欠損値の処理を行う。

- とりあえず、今回は欠損値を含むカラムは分析に使用しないようにする。

In [25]:
# I11からI14の列は使用しないので、トレーニングデータセットのカラムは"2:18"を使用する。

In [28]:
trainX = train.iloc[:, 2:18]
testX = test.iloc[:, 1:17]

## 性能指標を選択する。

- 性能指標は、評価関数「Logloss」を使用する。
 - 評価値は0以上の値をとり、精度が高いほど小さな値となる。

## 分類器と最適化アルゴリズムを選択する。
- ROC曲線やグリッドサーチを使用して、選択する。

一旦、決定木アルゴリズムを使用する。

## 最適化アルゴリズムを調整する。

グリッドサーチ+クロスバリデーションを使用して、決定木アルゴリズムを最適化する。

### 最適化後の決定木を描画してみる。

## モデルの性能を評価する。